# Scenario 1

In [2]:
import pandas as pd
import altair as alt
import numpy as np

In [37]:
def read_data(versuch:int):
    tx_logs = pd.read_csv(f"../logs/s1_v{versuch}_transmitter.csv", sep=';')
    rx_logs = pd.read_csv(f"../logs/s1_v{versuch}_receiver.csv", sep=';')
    
    tx_logs["ROLE"] = ["Tx"]*tx_logs.shape[0]
    rx_logs["ROLE"] = ["Rx"]*rx_logs.shape[0]
    
    tx_logs["EXPERIMENT"] = [versuch]*tx_logs.shape[0]
    rx_logs["EXPERIMENT"] = [versuch]*rx_logs.shape[0]
    
    return pd.concat([tx_logs, rx_logs], ignore_index=True)

## Experiment N

In [38]:
logs = pd.DataFrame([], columns=['SEQ', 'ACK', 'TIMESTAMP', 'PAYLOAD', 'RSSI', 'LQI', 'ROLE', 'EXPERIMENT'])

for experiment in range(1, 7):
    logs = pd.concat([logs, read_data(experiment)], ignore_index=True)

tx_logs = logs[logs["ROLE"]=="Tx"]
rx_logs = logs[logs["ROLE"]=="Rx"]

### Round Trip Times (RTT)

In [57]:
RTTs = pd.DataFrame((tx_logs[tx_logs["ACK"]==240].set_index(["SEQ", "EXPERIMENT"])["TIMESTAMP"] - tx_logs[tx_logs["ACK"]==0].set_index(["SEQ", "EXPERIMENT"])["TIMESTAMP"]))
RTTs.columns=["RTT"]

In [61]:
alt.Chart(RTTs.reset_index(), title="Round Trip Times in Scenario 1").mark_bar().encode(
    x="SEQ:O",
    y=alt.Y("RTT:Q", title="RTT (ms)"),
    row="EXPERIMENT:N"
)

alt.Chart(...)

### RSSI

In [63]:
RSSI_df = logs.loc[((logs["ACK"]==0) & (logs["ROLE"]=="Rx")) | ((logs["ACK"]==240) & (logs["ROLE"]=="Tx"))][["SEQ", "RSSI", "ROLE", "EXPERIMENT"]]

alt.Chart(RSSI_df, title=f"RSSI Values in Scenario 1 Experiment {experiment}").mark_point(filled=True).encode(
    x="SEQ:O",
    y=alt.Y("RSSI:Q", scale=alt.Scale(domain=(min(RSSI_df["RSSI"])-10, max(RSSI_df["RSSI"])+10))),
    color="ROLE:N",
    row="EXPERIMENT:N"
)

alt.Chart(...)

In [64]:
alt.Chart(RSSI_df, title=f"RSSI Variances Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="ROLE:N",
    y="variance(RSSI)",
    color="ROLE:N",
    column="EXPERIMENT:N"
)

alt.Chart(...)

### LQI

In [65]:
LQI_df = logs.loc[((logs["ACK"]==0) & (logs["ROLE"]=="Rx")) | ((logs["ACK"]==240) & (logs["ROLE"]=="Tx"))][["SEQ", "LQI", "ROLE", "EXPERIMENT"]]

alt.Chart(LQI_df, title=f"LQI Values in Scenario 1 Experiment {experiment}").mark_point(filled=True).encode(
    x="SEQ:O",
    y=alt.Y("LQI:Q", scale=alt.Scale(domain=(min(LQI_df["LQI"])-5, max(LQI_df["LQI"])+5))),
    color="ROLE:N",
    row="EXPERIMENT:N"
)

alt.Chart(...)

In [66]:
alt.Chart(LQI_df, title=f"LQII Variances Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="ROLE:N",
    y="variance(LQI)",
    color="ROLE:N",
    column="EXPERIMENT:N"
)

alt.Chart(...)

## Comparison

### 